<a href="https://colab.research.google.com/github/KevinArmbruster/ReinforcementLearning/blob/main/Atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade gym[all]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.5 MB/s 
     |████████████████████████████████| 3.4 MB 64.9 MB/s 
     |████████████████████████████████| 374 kB 75.5 MB/s 
     |████████████████████████████████| 18.3 MB 59.3 MB/s 
     |████████████████████████████████| 1.8 MB 67.4 MB/s 
     |████████████████████████████████| 2.4 MB 73.8 MB/s 
     |████████████████████████████████| 1.2 MB 57.7 MB/s 
     |████████████████████████████████| 296 kB 55.6 MB/s 
     |████████████████████████████████| 3.6 MB 63.3 MB/s 
     |████████████████████████████████| 207 kB 78.4 MB/s 
     |████████████████████████████████| 3.2 MB 62.9 MB/s 
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  A

In [1]:
from collections import deque, namedtuple

import numpy as np
import torch
import torch.nn as nn
import itertools
import gym
import matplotlib.pyplot as plt
from tqdm import trange

import gym
import ale_py

print('gym:', gym.__version__)
print('ale_py:', ale_py.__version__)

env = gym.make('Breakout-v0')

ModuleNotFoundError: ignored

In [2]:
class StateActionValueNetwork(nn.Module):

    def __init__(self, dim_states: int, n_actions: int, hidden_layer_sizes: list, lr: float, weight_decay: float = 0):
        super(StateActionValueNetwork, self).__init__()
        self.dim_states = dim_states
        self.n_actions = n_actions
        self.hidden_layer_sizes = hidden_layer_sizes
        self.layers = None
        self.activation_functions = None
        self.optimizer = None
        self.lr = lr
        self.max_grad_norm = 1  # advised range [0.5, 2]
        self.weight_decay = weight_decay

        self.setup_NN(dim_states, hidden_layer_sizes, n_actions)
        self.setup_optimizer()

    def setup_NN(self, dim_states, hidden_layer_sizes, n_actions, intermediate_af=nn.ReLU(), last_af=None):
        layers = nn.ModuleList()
        activation_functions = nn.ModuleList()

        nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),


            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,6)
        )

        self.layers = layers
        self.activation_functions = activation_functions

    def setup_optimizer(self):
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

    def forward(self, state: torch.Tensor):
        # out = torch.from_numpy(state)
        out = state

        for layer, af in zip(self.layers, self.activation_functions):
            if af:
                out = layer(out)
                out = af(out)
            else:
                out = layer(out)

        return out

    def backward(self, current, targets):
        # Training process, set gradients to 0
        self.optimizer.zero_grad()

        # Compute loss function
        loss = nn.functional.mse_loss(current, targets)

        # Compute gradient
        loss.backward()

        # Clip gradient norm to 1
        nn.utils.clip_grad_norm_(self.parameters(), max_norm=self.max_grad_norm)

        # Perform backward pass (backpropagation)
        self.optimizer.step()

        return (loss / len(current)).detach().numpy()


Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])


class ExperienceReplayBuffer(object):

    def __init__(self, maximum_length):
        self.buffer = deque(maxlen=int(maximum_length))

    def append(self, experience):
        self.buffer.append(experience)

    def __len__(self):
        return len(self.buffer)

    def sample_batch(self, n, combined=True):
        n = int(n)
        if n > len(self.buffer):
            raise IndexError('Tried to sample too many elements from the buffer!')

        if combined:
            n -= 1  # fetch 1 less
            indices = np.random.choice(len(self.buffer) - 1, size=n, replace=False)  # consider only rest of experiences
            batch = [self.buffer[i] for i in indices]
            batch.append(self.buffer[-1])  # append newest experience
        else:
            indices = np.random.choice(len(self.buffer), size=n, replace=False)
            batch = [self.buffer[i] for i in indices]

        # convert a list of tuples into a tuple of list we do zip(*batch)
        return zip(*batch)

In [7]:
env = gym.make("MountainCar-v0")
env.reset()

NameNotFound: ignored